お訓練の時間ですわよ

driveと連携します。
drive直下にdatasetというディレクトリ名で画像データがいっぱいあるとします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
!git clone https://github.com/laksjdjf/sd-trainer

In [ ]:
%cd sd-trainer

In [ ]:
# アプデ用
#!git pull origin main

In [ ]:
!pip install xformers -r requirements.txt

# データの前処理

データセットがある場所や前処理用のパス、使うモデルを指定してください。

In [7]:
import os

# 画像データがあるディレクトリのパスを指定してください。
dataset_path = "../dataset"

# 実際に訓練で使うデータセットのディレクトリパスを指定してください。
dataset_preprocess_path = "../dataset_preprocess/"

# モデルのパス（huggingfaceのモデル等）
model = "waifu-diffusion/wd-1-5-beta2"

## Aspect ratio bucketing

解像度の候補(bucket)をいくつか作ってそれにあわせます。

In [ ]:
!python3 preprocess/bucketing.py \
--input_dir {dataset_path} \
--output_dir {os.path.join(dataset_preprocess_path,"images")} \
--resolution 768 \
--min_length 512 \
--max_length 1024 \
--max_ratio 2 \
--threads 2

#メタデータを親ディレクトリに移動（ちょっとわかりづらいね）
!mv {os.path.join(dataset_preprocess_path,"images","buckets.json")} {dataset_preprocess_path}

## 潜在変数のキャッシュ

VAEによる潜在変数への変換結果をキャッシュします。

In [ ]:
!python preprocess/latent.py \
--directory {os.path.join(dataset_preprocess_path,"images")} \
--output_path {os.path.join(dataset_preprocess_path,"latents")} \
--model {model} \
--batch_size 4

## wd14taggerによるキャプションづけ

もっと高度なことしたい人は自分でやってcaptionsディレクトリに入れてください。ファイル名は画像と同じで、拡張子だけ.captionにしてください。

In [ ]:
#wd14-taggerを直下にダウンロードします。

from huggingface_hub import hf_hub_download
import os

DEFAULT_WD14_TAGGER_REPO = 'SmilingWolf/wd-v1-4-vit-tagger-v2'
TAGGER_DIR = 'wd-v1-4-vit-tagger-v2'
FILES = ["keras_metadata.pb", "saved_model.pb","selected_tags.csv"]
SUB_DIR = "variables"
SUB_DIR_FILES = ["variables.data-00000-of-00001", "variables.index"]

def download(path):
    model_dir = os.path.join(path, TAGGER_DIR)
    if not os.path.exists(model_dir):
        print(f"downloading wd14 tagger model from hf_hub. id: {DEFAULT_WD14_TAGGER_REPO}")
        for file in FILES:
            hf_hub_download(DEFAULT_WD14_TAGGER_REPO, file, cache_dir=model_dir, force_download=True, force_filename=file)
        for file in SUB_DIR_FILES:
            hf_hub_download(DEFAULT_WD14_TAGGER_REPO, file, subfolder=SUB_DIR, cache_dir=os.path.join(
                model_dir, SUB_DIR), force_download=True, force_filename=file)
    else:
        print("using existing wd14 tagger model")

download("./")

In [ ]:
#thresholdはタグ付けの閾値で、1に近づくほどタグが正確な代わりに少なくなり、0に近づけると不正確だがタグ数は多くなります。

!python preprocess/tagger.py \
--directory {os.path.join(dataset_preprocess_path,"images")} \
--output_path {os.path.join(dataset_preprocess_path,"captions")} \
--tagger_path {TAGGER_DIR} \
--make_caption \
--batch_size 4 \
--threshold 0.35

## pfgの特徴量を設定(任意)

pfgを学習したい場合は、pfgの特徴量をキャプションのところでもダウンロードしたwd14taggerで計算します。

In [ ]:
!python preprocess/create_pfg_feature.py \
--directory {os.path.join(dataset_preprocess_path,"images")} \
--output_path {os.path.join(dataset_preprocess_path,"pfg")} \
--batch_size 4 \
--threshold 0.35

## maskによる学習

顔部分だけマスクして学習したい場合は以下を実行してマスクデータをげっとしてください。

In [ ]:
!wget https://raw.githubusercontent.com/nagadomi/lbpcascade_animeface/master/lbpcascade_animeface.xml

In [ ]:
os.makedirs(os.path.join(dataset_preprocess_path,"mask"),exist_ok=True)

In [ ]:
!python preprocess/create_mask.py \
--path {os.path.join(dataset_preprocess_path,"images")} \
--output_path {os.path.join(dataset_preprocess_path,"mask")}

## ControlNetの学習

ControlNetを学習する際はControlNetに入力する画像が必要です。前処理はタスクごとに変わるので例はありません。基本的にはcontrolディレクトリにその画像を入れればいいわけですが、utils/dataset.py内のControlDatasetを書き換えるという手も一応あります。

# 学習設定

この訓練コードではyamlファイルで学習設定を管理していますが、colabでは編集しづらいので、元記事を参照してください。

今回はmaskを使いつつLoRAを学習してみます。

In [10]:
yaml = f'''
model: 
    input_path: {model}
    output_name: output
    v2: true #今のところ使わないのだが、一応
    v_prediction: true #SDv2(768)系のみtrue
    
dataset:
    module: utils.dataset.BaseDataset
    args:
        metadata: "buckets.json"
        path: "{dataset_preprocess_path}"
        mask: true
        pfg: false
        control: false
        prefix: "anime, "
    loader:
        module: torch.utils.data.DataLoader
        collate_fn: "identity"
        args:
            num_workers: 2
save:
    module: utils.save.Save
    args:
        wandb_name: null
        over_write: true
        save_n_epochs: 1
        save_n_steps: null
        image_logs: "image_logs"
        num_images: 4
        resolution: "640,896"
        prompt: null
        negative_prompt: "worst quality, low quality, medium quality, deleted, lowres, comic, bad anatomy,bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry"
        seed: 4545
        
train:
    train_unet: false
    train_encoder: false
    lr: "1e-4"
    lr_scheduler: "constant"
    epochs: 5
    batch_size: 2
    amp: "bfloat16"
    gradient_checkpointing: false
    use_xformers: true
    seed: 4545

feature:
    minibatch_repeat: 1
    up_only: false
    step_range: "0.0,1.0"
    test_steps: -1
    
optimizer:
    module: torch.optim.AdamW
    #args: 引数も指定できます。

network:
    module: networks.lora.LoRANetwork
    train: true
    resume: null
    args:
        rank: 16 # "dynamic"で動的に決定
        conv_rank: null # 指定するとloconになる
        module: null # "loha"でlohaになる
'''

with open("test.yaml","w") as f:
    f.write(yaml)

うおおおおお学習だ学習だ

モデルはtrained/の先で保存されます。
検証画像はimage_logsに保存されます。

In [ ]:
!python main.py test.yaml

生成はめんどうなのでありません。

おわり